In [1]:
import sys
sys.path.append("../")
from openai import OpenAI
from dotenv import load_dotenv
from pathlib import Path
import json
import os








In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)









In [3]:
input_path = Path("data/strategies_generated.json")

if input_path.exists():
    with open(input_path, "r") as f:
        raw_data = json.load(f)
        strategies = raw_data.get("strategies", [])
    print(f"✅ Loaded {len(strategies)} strategies to rank.")
else:
    print("❌ strategies_generated.json not found.")
    strategies = []









✅ Loaded 3 strategies to rank.


In [4]:
strategy_block = "\n\n".join(
    [f"{i+1}. {s['strategy_name']}\n- Description: {s['description']}\n- Why it fits: {s['why_it_fits']}\n- Best conditions: {s['best_conditions']}" for i, s in enumerate(strategies)]
)

prompt = f"""
You are an options strategy analyst. Rank the following strategies from most to least promising based on profit potential, trader alignment, and market adaptability.

Return JSON in this format:
{{
  "strategies": [
    {{
      "strategy_name": "...",
      "rank": 1,
      "reason": "..."
    }},
    ...
  ]
}}

Here are the strategies:
{strategy_block}
"""









In [5]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are an expert trading coach."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.5
)

ranking_output = response.choices[0].message.content.strip()
print("📊 GPT Ranking Response Preview:\n")
print(ranking_output[:1000])  # Preview only






📊 GPT Ranking Response Preview:

{
  "strategies": [
    {
      "strategy_name": "Diagonal Call Spread",
      "rank": 1,
      "reason": "This strategy has high profit potential, aligns well with the trader's understanding of time decay and use of options with different expiration dates, and is adaptable to a moderately bullish market."
    },
    {
      "strategy_name": "Iron Condor",
      "rank": 2,
      "reason": "Although this strategy has a broader range for the underlying asset price to move without impacting profitability, it is more reliant on a stable market with low volatility. It aligns with the trader's preference for strategies that profit from time decay."
    },
    {
      "strategy_name": "Long Straddle",
      "rank": 3,
      "reason": "This strategy has potential for high profits in volatile markets with large price movements. However, it is less adaptable to other market conditions and may not align as well with the trader's demonstrated skills and preferences

In [6]:
try:
    parsed = json.loads(ranking_output)
    ranked = parsed.get("strategies", [])
except json.JSONDecodeError:
    print("❌ Error decoding GPT output. Check formatting.")
    ranked = []

output_path = Path("data/ranked_strategies.json")
with open(output_path, "w") as f:
    json.dump({"strategies": ranked}, f, indent=2)

print(f"\n✅ Saved {len(ranked)} ranked strategies to {output_path}")



✅ Saved 3 ranked strategies to data/ranked_strategies.json
